In [49]:
import pandas as pd
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gensim
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import os
import random
import gensim.models.doc2vec
from wordcloud import WordCloud, STOPWORDS,ImageColorGenerator
from gensim.models.doc2vec import Doc2Vec
import numpy as np
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt 
import networkx as nx
from random import randint 
from itertools import count
import networkx as nx
import ast
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words
import openpyxl
import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math
import ast

import networkx as nx
from random import randint 
from itertools import count


In [40]:
year = '2014'

In [42]:
def convert_lower_case(data):
    return np.char.lower(data)

def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    # data = stemming(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    # data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data


def compute_similar_words(model,source_word, topn=5):
    similar_words = [source_word]
    try:
        top_words = model.wv.most_similar(source_word, topn=topn)
        similar_words.extend([val[0] for val in top_words])
    except KeyError as err:
        print(err.args)
    return similar_words    


In [43]:
df = pd.read_csv('../Entity_Recognition/Neighborhood_Separated_Articles/'+year+'.csv')
df = df.drop(['Unnamed: 0'], axis=1)

In [44]:
df = df.fillna("('no article', 'no_id')")
df['dorchester'] = df['dorchester'].apply(ast.literal_eval)
df['roxbury'] = df['roxbury'].apply(ast.literal_eval)
df['mattapan'] = df['mattapan'].apply(ast.literal_eval)
df['hyde_park'] = df['hyde_park'].apply(ast.literal_eval)
df['fenway'] = df['fenway'].apply(ast.literal_eval)
df['beacon_hill'] = df['beacon_hill'].apply(ast.literal_eval)
df['downtown'] = df['downtown'].apply(ast.literal_eval)
df['south_boston'] = df['south_boston'].apply(ast.literal_eval)
df['east_boston'] = df['east_boston'].apply(ast.literal_eval)
df['back_bay'] = df['back_bay'].apply(ast.literal_eval)
df['jamaica_plain'] = df['jamaica_plain'].apply(ast.literal_eval)
df['south_end'] = df['south_end'].apply(ast.literal_eval)
df['charlestown'] = df['charlestown'].apply(ast.literal_eval)
df['brighton'] = df['brighton'].apply(ast.literal_eval)
df['allston'] = df['allston'].apply(ast.literal_eval)
df['west_end'] = df['west_end'].apply(ast.literal_eval)
df['roslindale'] = df['roslindale'].apply(ast.literal_eval)
df['north_end'] = df['north_end'].apply(ast.literal_eval)
df['mission_hill'] = df['mission_hill'].apply(ast.literal_eval)
df['harbor_islands'] = df['harbor_islands'].apply(ast.literal_eval)
df['longwood_medical_area'] = df['longwood_medical_area'].apply(ast.literal_eval)
df['west_roxbury'] = df['west_roxbury'].apply(ast.literal_eval)

In [45]:
documents = {'hyde_park': [], 'beacon_hill': [], 'south_boston': [], 'jamaica_plain': [], 'east_boston': [],
                'south_end': [], 'back_bay': [], 'north_end': [], 'west_roxbury': [], 'mission_hill': [],
                'harbor_islands': [], 'west_end': [], 'longwood_medical_area': [],
                'dorchester': [], 'roxbury': [], 'downtown': [], 'fenway': [], 'mattapan': [], 'brighton': [],
                'charlestown': [], 'roslindale': [], 'allston': []}

for col in df.columns:
    tokens = []
    for i in range(df.shape[0]):
        article, _ = df.loc[i][col]
        if article != 'no article':
            text = word_tokenize(preprocess(article))
            tokens = tokens + text
    documents[col] = tokens
    #print(col)

subs = df.columns

In [46]:
spec_chars = ["!",'"',"#","%","&","'","(",")",
            "*","+",",","-",".","/",":",";","<",
            "=",">","?","@","[","\\","]","^","_",
            "`","{","|","}","~","–", "\xc2", "\xa0",
            "\x80", "\x9c", "\x99", "\x94", "\xad", "\xe2", "\x9d"]

df = pd.read_csv('globe_data/bostonglobe'+year+'.csv')
for char in spec_chars:
    df['text'] = df['text'].str.replace(char, ' ')

<ipython-input-46-071643416276>:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['text'] = df['text'].str.replace(char, ' ')


In [47]:
documents = []
for row in df.values:
    [row] = row
    temp = row.lower().split()
    documents.append(temp)

In [48]:
model = gensim.models.Word2Vec(window=10, min_count=2, sg=1, workers=10)
model.build_vocab(documents)  # prepare the model vocabulary
#Size = 20
model.train(corpus_iterable=documents, total_examples=len(documents), epochs=20)


(28540168, 36608460)

In [52]:
for col in subs:
    f1 = pd.read_csv('../Doc2Vec/Model/TFIDF_NeighborMostFreq/TFIDF_' + col + '.csv', nrows=21)
    keywords = f1['term'].tolist()

    topn = 5
    columns = ['word' + str(i - 1) for i in range(1, topn + 2)]
    df = pd.DataFrame(data=None, columns=columns)
    for source_word in keywords:
        similar_words = compute_similar_words(model, source_word, topn)
        df.loc[len(df)] = similar_words
    for i in range(1, len(similar_words)):
        sec_similar_words = compute_similar_words(model, similar_words[i], topn)
        df.loc[len(df)] = sec_similar_words
    df.to_csv('similar_words_task/TFIDF/similar_words_task_'+year+'.csv')


words = pd.read_csv('similar_words_task/TFIDF/similar_words_task_'+year+'.csv')

G = nx.Graph()
j = 1
for i, row in words.iterrows():
    for j in range(1, len(row)):
        G.add_node(i, label=row[j])
    for j in range(1, len(row)):
        G.add_edge(row[1], row[j])
remove = [node for node, degree in dict(G.degree()).items() if degree > 2]

pos = nx.spring_layout(G, k=0.3)
betCent = nx.betweenness_centrality(G, normalized=True, endpoints=True)
node_color = [20000.0 * G.degree(v) for v in G]
node_size =  [v * 10000 for v in betCent.values()]
plt.figure(figsize=(20, 20))
nx.draw_networkx(G, pos=pos, with_labels=True,
                 node_color=node_color,
                 node_size=node_size)
plt.axis('off')
plt.savefig('word_clouds_img/TFIDF/word_cloud_'+year+'.png')
plt.show()
    

("Key 'katahdin' not present",)


ValueError: cannot set a row with mismatched columns